In [ ]:
import cv2
import os
import shutil
import torch,torchvision
import math
import pandas
import natsort
import torchvision.transforms as transforms
from torchvision import datasets
import torch.optim as optim
import numpy as np
from PIL import Image, ImageFilter
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import torch
from torchsummary import summary
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/BinarySplit"

In [ ]:
path = path
class_dirs = os.listdir(path)

In [ ]:
def load_data(path, test_split, val_split, batch_size, input_size):
    modifyDict={'src':transforms.Compose([transforms.RandomHorizontalFlip(),
                                              transforms.Resize((224,224)),
                      transforms.RandomVerticalFlip(),
                      transforms.RandomRotation(15),
                      transforms.ToTensor(),
                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])}
    org_data = datasets.ImageFolder(root=path,transform =modifyDict['src'])
    testSubsetSize=int(len(org_data)*test_split)
    valDataSize=int(len(org_data)*val_split)
    trainSubsetSize=len(org_data)-valDataSize-testSubsetSize
    train_dataset,test_dataset,val_dataset = torch.utils.data.random_split(org_data,[trainSubsetSize,testSubsetSize,valDataSize])
    trainSetDataLoader = DataLoader(train_dataset,batch_size,shuffle=True)
    testSetDataLoader = DataLoader(test_dataset,batch_size,shuffle=False)
    valSetDataLoader = DataLoader(val_dataset,batch_size,shuffle=False)
    return trainSetDataLoader, testSetDataLoader, valSetDataLoader

In [ ]:
train_loader,test_loader,val_loader=load_data(path=path,
                                              test_split=0.1,val_split= 0.1,batch_size=32, input_size=[200,200])

In [ ]:
model = torchvision.models.mobilenet_v2(pretrained=False, num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cpu' if torch.cuda.is_available() else 'cuda')
model = model.to(device)

In [ ]:
flops_calc = 0
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        flops_calc += 2 * module.in_channels * module.out_channels * module.kernel_size[0] ** 2 * (224 // module.stride[0]) ** 2
    elif isinstance(module, torch.nn.Linear):
        flops_calc+= 2 * module.in_features * module.out_features

print(f"FLOPS: {flops_calc / 1e9:.3f} billion")

FLOPS: 4018.565 billion


In [ ]:
model = torchvision.models.resnet18(pretrained=False, num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cpu' if torch.cuda.is_available() else 'cuda')
model = model.to(device)

In [ ]:
flops_calc= 0
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        flops_calc += 2 * module.in_channels * module.out_channels * module.kernel_size[0] ** 2 * (224 // module.stride[0]) ** 2
    elif isinstance(module, torch.nn.Linear):
        flops_calc += 2 * module.in_features * module.out_features

print(f"FLOPS: {flops_calc / 1e9:.3f} billion")

FLOPS: 990.436 billion


In [ ]:
model = torchvision.models.shufflenet_v2_x1_0(pretrained=False, num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cpu' if torch.cuda.is_available() else 'cuda')
model = model.to(device)

In [ ]:
flops_calc = 0
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        flops_calc += 2 * module.in_channels * module.out_channels * module.kernel_size[0] ** 2 * (224 // module.stride[0]) ** 2
    elif isinstance(module, torch.nn.Linear):
        flops_calc += 2 * module.in_features * module.out_features

print(f"FLOPS: {flops_calc / 1e9:.3f} billion")

FLOPS: 393.202 billion
